In [1]:
!pip install pandas numpy langdetect

import pandas as pd
import numpy as np
from langdetect import detect


In [2]:
sentiment_df = pd.read_csv('airline_sentiment.csv')

passenger_df = pd.read_csv('passenger_satisfaction.csv')

expedia_df = pd.read_csv('expedia_dataset.csv')


In [3]:
skytrax_df = pd.read_csv('skytrax_reviews.csv')

In [4]:
print(skytrax_df.columns)
print(skytrax_df.head)

Index(['airline', 'overall', 'author', 'review_date', 'customer_review',
       'aircraft', 'traveller_type', 'cabin', 'route', 'date_flown',
       'seat_comfort', 'cabin_service', 'food_bev', 'entertainment',
       'ground_service', 'value_for_money', 'recommended'],
      dtype='object')
<bound method NDFrame.head of                       airline  overall               author  \
0                         NaN      NaN                  NaN   
1            Turkish Airlines      7.0  Christopher Hackley   
2                         NaN      NaN                  NaN   
3            Turkish Airlines      2.0        Adriana Pisoi   
4                         NaN      NaN                  NaN   
...                       ...      ...                  ...   
131890  Ukraine International      NaN     Andriy Yesypenko   
131891                    NaN      NaN                  NaN   
131892  Ukraine International      NaN    Volodya Bilotkach   
131893                    NaN      NaN         

In [5]:
print(expedia_df.columns)
print(expedia_df.head)

Index(['Unnamed: 0', 'date_time', 'site_name', 'posa_continent',
       'user_location_country', 'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster'],
      dtype='object')
<bound method NDFrame.head of        Unnamed: 0            date_time  site_name  posa_continent  \
0        24636210  2014-11-03 16:02:28         24               2   
1        19837144  2013-03-13 19:25:01         11               3   
2        13066459  2014-10-13 13:20:25          2               3   
3         4691082  2013-11-05 10:40:34         11               3   
4         4878884  2014-06-10 13:34:56          2               3   
...           ...                  ...        ...          

In [6]:
print(passenger_df.columns)
print(passenger_df.head)

Index(['ID', 'Gender', 'Age', 'Customer Type', 'Type of Travel', 'Class',
       'Flight Distance', 'Departure Delay', 'Arrival Delay',
       'Departure and Arrival Time Convenience', 'Ease of Online Booking',
       'Check-in Service', 'Online Boarding', 'Gate Location',
       'On-board Service', 'Seat Comfort', 'Leg Room Service', 'Cleanliness',
       'Food and Drink', 'In-flight Service', 'In-flight Wifi Service',
       'In-flight Entertainment', 'Baggage Handling', 'Satisfaction'],
      dtype='object')
<bound method NDFrame.head of             ID  Gender  Age Customer Type Type of Travel         Class  \
0            1    Male   48    First-time       Business      Business   
1            2  Female   35     Returning       Business      Business   
2            3    Male   41     Returning       Business      Business   
3            4    Male   50     Returning       Business      Business   
4            5  Female   49     Returning       Business      Business   
...      

In [7]:
print(sentiment_df.columns)
print(sentiment_df.head)

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')
<bound method NDFrame.head of                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  56958737169335500

In [8]:
skytrax_df.info()
sentiment_df.info()
passenger_df.info()
expedia_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131895 entries, 0 to 131894
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          65947 non-null  object 
 1   overall          64017 non-null  float64
 2   author           65947 non-null  object 
 3   review_date      65947 non-null  object 
 4   customer_review  65947 non-null  object 
 5   aircraft         19718 non-null  object 
 6   traveller_type   39755 non-null  object 
 7   cabin            63303 non-null  object 
 8   route            39726 non-null  object 
 9   date_flown       39633 non-null  object 
 10  seat_comfort     60681 non-null  float64
 11  cabin_service    60715 non-null  float64
 12  food_bev         52608 non-null  float64
 13  entertainment    44193 non-null  float64
 14  ground_service   39358 non-null  float64
 15  value_for_money  63975 non-null  float64
 16  recommended      64440 non-null  object 
dtypes: float64

In [9]:
# # Keep relevant columns
# skytrax_df = skytrax_df[['airline', 'author', 'review_date', 'customer_review', 'traveller_type', 'cabin', 'recommended']]

# # Drop NaN reviews
# skytrax_df = skytrax_df[skytrax_df['customer_review'].notnull()].reset_index(drop=True)

# # Assign synthetic user_id
# skytrax_df['user_id'] = ['user_' + str(i) for i in range(len(skytrax_df))]


In [10]:
skytrax_df = skytrax_df[['airline', 'customer_review', 'recommended']]
skytrax_df = skytrax_df.dropna()
skytrax_df = skytrax_df[skytrax_df['recommended'].isin(['yes', 'no'])].reset_index(drop=True)

skytrax_df['sentiment'] = skytrax_df['recommended'].map({'yes': 1, 'no': 0})

skytrax_df['language'] = skytrax_df['customer_review'].apply(lambda x: detect(str(x)))

skytrax_df['user_id'] = ['user_' + str(i) for i in range(len(skytrax_df))]



In [11]:
passenger_df = passenger_df.rename(columns={
    'ID': 'user_id', 'Gender': 'gender', 'Age': 'age', 'Class': 'travel_class', 'Satisfaction': 'satisfaction'
})
passenger_df = passenger_df[['user_id', 'gender', 'age', 'travel_class', 'satisfaction']]

passenger_df['user_id'] = ['user_' + str(i) for i in range(len(passenger_df))]


In [12]:
expedia_df = expedia_df[['user_id', 'date_time', 'is_booking']]

# Convert user_id to synthetic pattern
expedia_df['user_id'] = ['user_' + str(i) for i in range(len(expedia_df))]

# Drop duplicates (latest session only)
expedia_df = expedia_df.drop_duplicates(subset='user_id', keep='last')


In [13]:
min_len = min(len(skytrax_df), len(passenger_df), len(expedia_df))

skytrax_df = skytrax_df.sample(n=min_len, random_state=42).reset_index(drop=True)
passenger_df = passenger_df.sample(n=min_len, random_state=42).reset_index(drop=True)
expedia_df = expedia_df.sample(n=min_len, random_state=42).reset_index(drop=True)

# Now all have the same 'user_id' order
skytrax_df['user_id'] = ['user_' + str(i) for i in range(min_len)]
passenger_df['user_id'] = ['user_' + str(i) for i in range(min_len)]
expedia_df['user_id'] = ['user_' + str(i) for i in range(min_len)]


In [14]:
# Merge Skytrax + Passenger
merge1 = pd.merge(skytrax_df, passenger_df, on='user_id', how='inner')

# Merge with Expedia
master_df = pd.merge(merge1, expedia_df, on='user_id', how='inner')


In [15]:
master_df.head()


,airline,customer_review,recommended,sentiment,language,user_id,gender,age,travel_class,satisfaction,date_time,is_booking
0,Aer Lingus,DUB-MAL. Flew to Malaga recently and overall v...,yes,1,en,user_0,Male,23,Economy,Neutral or Dissatisfied,2014-06-12 17:49:40,0
1,Thai Airways,TG 911 30th Jan 2015. Inbound flight TG916 on ...,no,0,en,user_1,Male,38,Economy,Satisfied,2014-01-14 22:43:20,0
2,United Airlines,âœ… Trip Verified | Columbia to Tokyo via Denv...,no,0,en,user_2,Male,33,Business,Satisfied,2014-07-11 10:58:21,0
3,flydubai,Flydubai to and from Port Sudan. Both flights ...,yes,1,en,user_3,Male,68,Economy,Satisfied,2013-09-17 05:06:11,0
4,Etihad Airways,Flew MEL-AUH-DUB. Booked business and got unex...,yes,1,en,user_4,Male,27,Business,Satisfied,2014-07-09 17:55:09,1


In [32]:
print("Rows, Columns:", master_df.shape)


Rows, Columns: (64440, 12)


In [34]:
print(master_df.columns.tolist())


['airline', 'customer_review', 'recommended', 'sentiment', 'language', 'user_id', 'gender', 'age', 'travel_class', 'satisfaction', 'date_time', 'is_booking']


In [36]:
# Save the master dataframe to CSV with UTF-8 encoding
master_df.to_csv("master_airline_dataset.csv", index=False, encoding='utf-8')


In [38]:
master_df['language'].value_counts()


language
en    64427
es        4
fr        4
nl        3
de        1
it        1
Name: count, dtype: int64